# File "Network"

In [39]:
"""""""""""""""""""""""""""""""""""""""
network.py
Модуль создания и обучения нейронной сети для распознавания рукописных цифр
с использованием метода градиентного спуска.
Группа:ИСТб-22-1
ФИО:Моргунов Александр Алексеевич
"""""""""""""""""""""""""""""""""""""""
#### Библиотеки
# Стандартные библиотеки
import random # библиотека функций для генерации случайных значений
# Сторонние библиотеки
import numpy as np # библиотека функций для работы с матрицами
""" ---Раздел описаний--- """
""" --Описание класса Network--"""
class Network(object): # используется для описания нейронной сети
    def __init__(self, sizes): # конструктор класса
        # self – указатель на объект класса
        # sizes – список размеров слоев нейронной сети
        self.num_layers = len(sizes) # задаем количество слоев нейронной сети
        self.sizes = sizes # задаем список размеров слоев нейронной сети
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]] # задаем случайные начальные смещения
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1],sizes[1:])] # задаем случайные начальные веса связей
    def feedforward(self, a):
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a
    def SGD( # Стохастический градиентный спуск
    self # указатель на объект класса
    , training_data # обучающая выборка
    , epochs # количество эпох обучения
    , mini_batch_size # размер подвыборки
    , eta # скорость обучения
    , test_data # тестирующая выборка
    ):
        test_data = list(test_data) # создаем список объектов тестирующей выборки
        n_test = len(test_data) # вычисляем длину тестирующей выборки
        training_data = list(training_data) # создаем список объектов обучающей выборки
        n = len(training_data) # вычисляем размер обучающей выборки
        for j in range(epochs): # цикл по эпохам
            random.shuffle(training_data) # перемешиваем элементы обучающей выборки
            mini_batches = [training_data[k:k+mini_batch_size] for k in range(0, n, mini_batch_size)] # создаем подвыборки
            for mini_batch in mini_batches: # цикл по подвыборкам
                self.update_mini_batch(mini_batch, eta) # один шаг градиентного спуска
            print ("Epoch {0}: {1} / {2} = {3}".format(j, self.evaluate(test_data), n_test, self.evaluate(test_data)/n_test)) # смотрим прогресс в обучении
    def update_mini_batch( # Шаг градиентного спуска
        self # указатель на объект класса
        , mini_batch # подвыборка
        , eta # скорость обучения
        ):
        nabla_b = [np.zeros(b.shape) for b in self.biases] # список градиентов dC/db для каждого слоя (первоначально заполняются нулями)
        nabla_w = [np.zeros(w.shape) for w in self.weights] # список градиентов dC/dw для каждого слоя (первоначально заполняются нулями)
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y) # послойно вычисляем градиенты dC/db и dC/dw для текущего прецедента (x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)] # суммируем градиенты dC/db для различных прецедентов текущей подвыборки
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)] # суммируем градиенты dC/dw для различных прецедентов текущей подвыборки
            self.weights = [w-(eta/len(mini_batch))*nw for w, nw in zip(self.weights, nabla_w)] # обновляем все веса w нейронной сети
            self.biases = [b-(eta/len(mini_batch))*nb for b, nb in zip(self.biases, nabla_b)] # обновляем все смещения b нейронной сети
    def backprop( # Алгоритм обратного распространения
        self # указатель на объект класса
        , x # вектор входных сигналов
        , y # ожидаемый вектор выходных сигналов
        ):
        nabla_b = [np.zeros(b.shape) for b in self.biases] # список градиентов dC/db для каждого слоя (первоначально заполняются нулями)
        nabla_w = [np.zeros(w.shape) for w in self.weights] # список градиентов dC/dw для каждого слоя (первоначально заполняются нулями)
        # определение переменных
        activation = x # выходные сигналы слоя (первоначально соответствует выходным сигналам 1-го слоя или входным сигналам сети)
        activations = [x] # список выходных сигналов по всем слоям (первоначально содержит только выходные сигналы 1-го слоя)
        zs = [] # список активационных потенциалов по всем слоям (первоначально пуст)

        # прямое распространение
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b # считаем активационные потенциалы текущего слоя
            zs.append(z) # добавляем элемент (активационные потенциалы слоя) в конец списка
            activation = sigmoid(z) # считаем выходные сигналы текущего слоя, применяя сигмоидальную функцию активации к активационным потенциалам слоя
            activations.append(activation) # добавляем элемент (выходные сигналы слоя) в конец списка 
        # обратное распространение
        delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1]) # считаем меру влияния нейронов выходного слоя L на величину ошибки (BP1)
        nabla_b[-1] = delta # градиент dC/db для слоя L (BP3)
        nabla_w[-1] = np.dot(delta, activations[-2].transpose()) # градиент dC/dw для слоя L (BP4)

        for l in range(2, self.num_layers):
            z = zs[-l] # активационные потенциалы l-го слоя (двигаемся по списку справа налево)
            sp = sigmoid_prime(z) # считаем сигмоидальную функцию от активационных потенциалов l-го слоя
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp # считаем меру влияния нейронов l-го слоя на величину ошибки (BP2)
            nabla_b[-l] = delta # градиент dC/db для l-го слоя (BP3)
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())# градиент dC/dw для l-го слоя (BP4)
        return (nabla_b, nabla_w)
    def evaluate(self, test_data): # Оценка прогресса в обучении
        test_results = [(np.argmax(self.feedforward(x)), y) for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)
    def cost_derivative(self, output_activations, y): # Вычисление частных производных стоимостной функции по выходным сигналам последнего слоя
        return (output_activations-y)
""" --Конец описания класса Network--"""
""" --- Конец раздела описаний--- """
""" ---Тело программы--- """
net = Network([2, 3, 1]) # создаем нейронную сеть из трех слоев
""" ---Конец тела программы--- """
""" Вывод результата на экран: """
print('Сеть net:')
print('Количетво слоев:', net.num_layers)
for i in range(net.num_layers):
    print('Количество нейронов в слое', i,':',net.sizes[i])
for i in range(net.num_layers-1):
    print('W_',i+1,':')
    print(np.round(net.weights[i],2))
    print('b_',i+1,':')
    print(np.round(net.biases[i],2))

Сеть net:
Количетво слоев: 3
Количество нейронов в слое 0 : 2
Количество нейронов в слое 1 : 3
Количество нейронов в слое 2 : 1
W_ 1 :
[[-1.07 -1.18]
 [-0.07  1.6 ]
 [-0.62  0.32]]
b_ 1 :
[[ 1.47]
 [-1.49]
 [-2.4 ]]
W_ 2 :
[[ 0.27 -1.3  -0.21]]
b_ 2 :
[[1.18]]


In [40]:
def sigmoid(z): # определение сигмоидальной функции активации
    return 1.0/(1.0+np.exp(-z))

In [41]:
def sigmoid_prime(z):# Производная сигмоидальной функции
    return sigmoid(z)*(1-sigmoid(z))

# File "MNIST loader"

In [42]:
"""
mnist_loader.py
~~~~~~~~~~
Модуль для подключения и использования базы данных MNIST.
Группа:ИСТб-22-1
ФИО:Моргунов Александр Алексеевич
"""
import gzip # библиотека для сжатия и распаковки файлов gzip и gunzip.
import pickle # библиотека для сохранения и загрузки сложных объектов Python.
import numpy as np # библиотека для работы с матрицами
def load_data():
    f = gzip.open('mnist.pkl.gz', 'rb') # открываем сжатый файл gzip в двоичном режиме
    training_data, validation_data, test_data = pickle.load(f, encoding='latin1') # загружам таблицы из файла
    f.close() # закрываем файл
    return (training_data, validation_data, test_data)

In [43]:
def load_data_wrapper():
    tr_d, va_d, te_d = load_data() # инициализация наборов данных в формате MNIST
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]] # преобразование массивов размера 1 на 784 к массивам размера 784 на 1
    training_results = [vectorized_result(y) for y in tr_d[1]] # представление цифр от 0 до 9 в виде массивов размера 10 на 1
    training_data = zip(training_inputs, training_results) # формируем набор обучающих данных из пар (x, y)
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]] # преобразование массивов размера 1 на 784 к массивам размера 784 на 1
    validation_data = zip(validation_inputs, va_d[1]) # формируем набор данных проверки из пар (x, y)
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]] # преобразование массивов размера 1 на 784 к массивам размера 784 на 1
    test_data = zip(test_inputs, te_d[1]) # формируем набор тестовых данных из пар (x, y)
    return (training_data, validation_data, test_data)

In [44]:
def vectorized_result(j):
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

# Interpretator

In [45]:
training_data, validation_data, test_data = load_data_wrapper()

In [46]:
net = Network([784, 30, 10])

In [47]:
net.SGD(training_data, 30, 10, 3.0, test_data=test_data)

Epoch 0: 7575 / 10000 = 0.7575
Epoch 1: 7471 / 10000 = 0.7471
Epoch 2: 8680 / 10000 = 0.868
Epoch 3: 8537 / 10000 = 0.8537
Epoch 4: 8687 / 10000 = 0.8687
Epoch 5: 8541 / 10000 = 0.8541
Epoch 6: 8810 / 10000 = 0.881
Epoch 7: 8673 / 10000 = 0.8673
Epoch 8: 8860 / 10000 = 0.886
Epoch 9: 8643 / 10000 = 0.8643
Epoch 10: 8763 / 10000 = 0.8763
Epoch 11: 8939 / 10000 = 0.8939
Epoch 12: 8816 / 10000 = 0.8816
Epoch 13: 8842 / 10000 = 0.8842
Epoch 14: 8797 / 10000 = 0.8797
Epoch 15: 8721 / 10000 = 0.8721


C:\Users\morgu\AppData\Local\Temp\ipykernel_9260\3245411285.py:2: RuntimeWarning: overflow encountered in exp
  return 1.0/(1.0+np.exp(-z))


Epoch 16: 8932 / 10000 = 0.8932
Epoch 17: 8924 / 10000 = 0.8924
Epoch 18: 8942 / 10000 = 0.8942
Epoch 19: 8924 / 10000 = 0.8924
Epoch 20: 8919 / 10000 = 0.8919
Epoch 21: 8884 / 10000 = 0.8884
Epoch 22: 8949 / 10000 = 0.8949
Epoch 23: 9019 / 10000 = 0.9019
Epoch 24: 8837 / 10000 = 0.8837
Epoch 25: 8881 / 10000 = 0.8881
Epoch 26: 9079 / 10000 = 0.9079
Epoch 27: 9026 / 10000 = 0.9026
Epoch 28: 8948 / 10000 = 0.8948
Epoch 29: 8862 / 10000 = 0.8862


# File "Network 2"

In [ ]:
"""""""""""""""""""""""""""""""""""""""
network2.py
Модуль создания и обучения нейронной сети для распознавания рукописных цифр
на основе метода стохастического градиентного спуска для прямой нейронной
сети и стоимостной функции на основе перекрестной энтропии, регуляризации и
улучшеннного способа инициализации весов нейронной сети.
Группа: ИСТб-22-1
ФИО: Моргунов Александр Алексеевич
"""""""""""""""""""""""""""""""""""""""
#### Библиотеки
# Стандартные библиотеки
import json # библиотека для кодирования/декодирования данных/объектов Python
import random # библиотека функций для генерации случайных значений
import sys # библиотека для работы с переменными и функциями, имеющими отношение к интерпретатору и его окружению
# Сторонние библиотеки
import numpy as np # библиотека функций для работы с матрицами
""" ---Раздел описаний--- """

""" -- Определение стоимостных функции --"""
class QuadraticCost(object): # Определение среднеквадратичной стоимостной функции
    @staticmethod
    def fn(a, y): # Cтоимостная функция
        return 0.5*np.linalg.norm(a-y)**2
    @staticmethod
    def delta(z, a, y): # Мера влияния нейронов выходного слоя на величину ошибки
        return (a-y) * sigmoid_prime(z)
""" -- Конец описания стоимостной функции -- """
""" -- Описание класса стоимостной функции на основе перекрестной энтропии -- """
class CrossEntropyCost(object): # Определение стоимостной функции на основе перекрестной энтропии
    @staticmethod
    def fn(a, y): # Cтоимостная функция
        return np.sum(np.nan_to_num(-y*np.log(a)-(1-y)*np.log(1-a)))
    @staticmethod
    def delta(z, a, y): # Мера влияния нейронов выходного слоя на величину ошибки
        return (a-y)
""" -- Конец описания класса стоимостной функции на основе перекрестной энтропии -- """
""" --Описание класса Network-- """
class Network(object):
    def __init__( # конструктор класса
    self # указатель на объект класса
    , sizes # список размеров слоев нейронной сети
    , cost=CrossEntropyCost # стоимостная функция (по умолчанию будет использоваться функция перекрестной энтропии)
    ):
    self.num_layers = len(sizes) # задаем количество слоев нейронной сети
    self.sizes = sizes # задаем список размеров слоев нейронной сети
    self.default_weight_initializer() # метод инициализации начальных весов связей и смещений по умолчанию
    self.cost=cost # задаем стоимостную функцию
    def default_weight_initializer(self): # метод инициализации начальных весов связей и смещений
        self.biases = [np.random.randn(y, 1) for y in self.sizes[1:]] # задаем случайные начальные смещения
        self.weights = [np.random.randn(y, x)/np.sqrt(x) for x, y in zip(self.sizes[:-1], self.sizes[1:])] # задаем случайные начальные веса связей
    def large_weight_initializer(self):
        self.biases = [np.random.randn(y, 1) for y in self.sizes[1:]] # задаем случайные начальные смещения
        self.weights = [np.random.randn(y, x) for x, y in zip(self.sizes[:-1], self.sizes[1:])] # задаем случайные начальные веса
    def feedforward(self, a):
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a
""" -- Конец описания класса Network -- """        
""" --- Конец раздела описаний--- """

